In [1]:
import os

KAGGLE =  os.getenv("KAGGLE_URL_BASE") is not None
COLAB = os.getenv("COLAB_GPU") is not None
TPU = os.getenv("XRT_TPU_CONFIG") is not None
LOCAL = not KAGGLE and not COLAB

if not LOCAL:
    !git clone https://github.com/nclibz/MRKnee/

if COLAB:
    os.chdir('/content/MRKnee/')
    !git checkout v3
    from google.colab import drive
    drive.mount('/content/drive')
    DATADIR = "/content/drive/MyDrive/MRKnee/data"
    if TPU:
        !pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

if KAGGLE:
    os.chdir('/kaggle/working/MRKnee/')
    !git checkout v3
    DATADIR = "/kaggle/input/mrknee/MRNet"
    
    if TPU:
        !pip install torchtext==0.9
        !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
        !python pytorch-xla-env-setup.py --version 1.8

if not LOCAL:
    !pip install --quiet "pytorch-lightning>=1.4.9" "torchmetrics>=0.5" "timm" "neptune-client" "optuna" "PyMySql" "torch-tb-profiler"
    !pip install albumentations --upgrade --quiet
    BACKBONE = "tf_efficientnetv2_s_in21k"

if LOCAL:
    DATADIR = "data"
    BACKBONE = 'mobilenetv3_large_100_miil_in21k'
    %load_ext autoreload
    %autoreload 2

In [2]:
DIAGNOSIS = "acl"
PLANE = "sagittal"

In [3]:
from src.study import Study
from src.model import MRKnee
from src.data import MRKneeDataModule
from src.augmentations import Augmentations
from src.callbacks import Callbacks
from src.cfg import Cfg
import pytorch_lightning as pl

pl.seed_everything(123)

Global seed set to 123


123

In [7]:

def objective(trial, diagnosis=DIAGNOSIS, plane=PLANE, backbone=BACKBONE, datadir=DATADIR, profile = False):

    model = MRKnee(
        backbone=backbone,
        drop_rate=trial.suggest_int("drop_rate", 30, 80, step = 5) / 100,
        final_drop=trial.suggest_int("final_drop", 0, 30, step = 5) / 100,
        learning_rate=trial.suggest_loguniform('lr', 1e-6, 1e-3),
        log_auc=True,
        log_ind_loss=False,
        adam_wd=trial.suggest_loguniform('adam_wd', 0.001, 0.1),
        max_epochs=20,
        precision=32,
    )

    augs = Augmentations(
        model,
        shift_limit=trial.suggest_int("shift_limit", 0, 15) / 100,
        scale_limit=trial.suggest_int("scale_limit", 0, 15) / 100,
        rotate_limit=trial.suggest_int("rotate_limit", 0, 15) / 100,
        reverse_p=0.5,
        indp_normalz=True,
    )

    dm = MRKneeDataModule(
        datadir=datadir,
        diagnosis=diagnosis,
        plane=plane,
        transforms=augs,
        clean=True,
        num_workers=2,
        pin_memory=True,
        trim_train=True,
    )

    configs = Cfg(model = model, dm = dm, augs = augs)
    cfg = configs.get_cfg()
    
    if trial is not None:
        callbacks = Callbacks(cfg, trial, neptune_name="mrkneev3")
        neptune_logger = callbacks.get_neptune_logger()
        list_of_cbs = callbacks.get_callbacks()
        fast_dev_run = False
    else:
        neptune_logger = False
        list_of_cbs = None
        fast_dev_run = 50

    profiler = pl.profiler.PyTorchProfiler(dirpath = "src/logs", filename = "profiler") if profile else False
        


    trainer = pl.Trainer(
        gpus=1,
        precision=cfg["precision"],
        max_epochs=cfg["max_epochs"],
        logger=neptune_logger,
        log_every_n_steps=100,
        num_sanity_val_steps=0,
        callbacks=list_of_cbs,
        progress_bar_refresh_rate=20,
        deterministic=True,
        profiler = profiler, 
        fast_dev_run = fast_dev_run,
    )

    trainer.fit(model, dm)

    ## UPLOAD BEST CHECKPOINTS TO LOG
    if trial is not None:
        callbacks.upload_best_checkpoints()

    return callbacks.model_checkpoint.best_model_score.item()


In [5]:
# For testing
#objective(trial = None)

study = Study(diagnosis = DIAGNOSIS,
              plane = PLANE,
              backbone=BACKBONE,
              min_epochs=10)

/home/nicolai/miniconda3/envs/dl/lib/python3.9/site-packages/optuna/samplers/_tpe/sampler.py:263: ExperimentalWarning: ``multivariate`` option is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2021-10-08 14:49:20,589] A new study created in RDB with name: acl_sagittal_mobilenetv3_large_100_miil_in21k


In [ ]:

study.optimize(objective, n_trials=40, n_trials = 20)